This notebook cleans and combines the input bash script and composition list data file to provide a complete set of the input parameters sampled in the sensitivity analysis.

The file it saves is large (over 4GB), with 824000 lines.  CSV entries in that file are as follows:  
End time [s], Tmax [K], h [c/min], Carbon [mole fraction], Hydrogen [mole fraction], Oxygen [mole fraction], followed by 406 rate constant adjustment factors for all the reactions in the model.

In [3]:
import csv

# Read the bash script and store the lines that include simulation calls
params = []
with open('sa_data/sa_bash_script.sh', 'rb') as script:
    for line in script:
        if line.startswith('mkdir'):
            params.append(line.split(';')[2].split('&')[0])

# Read the composition list to get the equivalent composition of each line
# compositions are mole fractions of C, H, O
composition = []
with open('sa_data/SAcompositionlist.dat', 'rb') as comp:
    for line in comp:
        if line.startswith('param'):
            line = line.rstrip()
            composition.append(line.split('\t')[1:])

In [8]:
# Write the cleaned set of parameters to a csv file
with open('input_parameters.csv', 'wb') as paramfile:
    csvwriter = csv.writer(paramfile, delimiter=',')
    
    for i,line in enumerate(params):
        end_time_s = line.split(' ')[3]
        h_cmin = line.split(' ')[7]
        tmax_K = line.split(' ')[8]
        rate_const_adjust = line.split(' ')[15].split(',')
        comp = composition[i]
        # put all these together into a new parameter line
        csvwriter.writerow([end_time_s] + [tmax_K] + [h_cmin]
                          + comp + rate_const_adjust)
    